In [1]:
import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
transform = transforms.Compose([transforms.ToTensor()])
trainset_tmp = datasets.MNIST('~/', download=True, train=True, transform=transform)
trainloader_tmp = torch.utils.data.DataLoader(trainset_tmp, batch_size=10_000, shuffle=True)
testset = datasets.MNIST('~/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=512, shuffle=False)

In [2]:
def data_divides__minst():
    Train = []
    for image in trainset_tmp:
        for i in range(10):
            if image[1] ==i:
                Ima = image[0][0].reshape(1,784)
                number = np.array([0 for i in range(10)])
                number[i] = 1
                Train.append([np.array(Ima[0][:]),number[:]])
    return Train
Train = data_divides__minst()

def data_divides__minst_test():
    Test = []
    for image in testset:
        for i in range(10):
            if image[1] ==i:
                Ima = image[0][0].reshape(1,784)
                number = np.array([0 for i in range(10)])
                number[i] = 1
                Test.append([np.array(Ima[0][:]),number[:]])
    return Test
Test = data_divides__minst_test()
#训练集测试集合处理为球面数据
Test_data = []
for image in Test:
    image_pic = image[0]/10
    a = 2/(sum(image_pic*image_pic)+1)
    Ima = np.append(image_pic*a, a-1)
    Test_data.append([Ima[:],image[1]])

In [5]:
import numpy as np
import math
import random
import time
class T_kernel_SGD():
    def __init__(self, d, gamma_0, theta, t, s,N):
        self.d = d
        self.s = s
        self.gamma_0 = gamma_0
        self.theta = theta
        self.t = t
        self.f_hat = [0]
        self.f_bar = [0]
        self.sgd_f = [0 for k in range(self.d)]
        self.Z_0 = np.array(1)
        self.Z_1 = self.d*np.ones(self.d)
        self.Z_2 = [-(self.d+2)/2,np.zeros(self.d),np.ones((self.d,self.d))*(self.d*(self.d+2))/2]
        self.Znt = [self.Z_0]
        self.ZntX = []
        self.order = 0
        self.dimPiLn = 1
        self.N = N
        self.F_hat =[self.f_hat[:] for i in range(10)]
        self.F_bar =[self.f_hat[:] for i in range(10)]
        
    
    def update_Z_nt(self):
        if self.order == 1:
            coe = 1/(self.d+1)**(2*self.s)
            self.Znt.append(coe*self.Z_1)
        elif self.order == 2:
            coe = 1/(self.d*(self.d+3)/2)**(2*self.s)
            self.Znt.append(np.zeros((self.d,self.d)))
            self.Znt = [a+coe*b for a,b in zip(self.Znt,self.Z_2)]
    
    def compute_Znt(self,z):
        if self.order == 0:
            self.ZntX = [self.Znt[0]]
        elif self.order == 1:
            self.ZntX = [self.Znt[0]]
            self.ZntX.append(z*self.Znt[1])
        elif self.order == 2:
            self.ZntX = [self.Znt[0]]
            self.ZntX.append(z*self.Znt[1])
            ZZ = np.dot(np.expand_dims(z,1),np.expand_dims(z,0))
            self.ZntX.append(ZZ*self.Znt[2])
            
    def comput_F_bar(self,x):
        Y = np.zeros(10)
        for i in range(10):
            if self.order == 0:
                y = self.F_bar[i][0]
            elif self.order == 1:
                y = self.F_bar[i][0] + np.dot(x,self.F_bar[i][1])
            elif self.order == 2:
                y = self.F_bar[i][0] + np.dot(x,self.F_bar[i][1]) + np.dot(x,np.dot(self.F_bar[i][2],x))
            Y[i] = y
        return Y
    
    def comput_F_hat(self,x):
        Y = np.zeros(10)
        for i in range(10):
            if self.order == 0:
                y = self.F_hat[i][0]
            elif self.order == 1:
                y = self.F_hat[i][0] + np.dot(x,self.F_hat[i][1])
            elif self.order == 2:
                y = self.F_hat[i][0] + np.dot(x,self.F_hat[i][1]) + np.dot(x,np.dot(self.F_hat[i][2],x))
            Y[i] = y
        return Y
    
    def comput_f_sgd(self,x):
        y = sum([j*k for j,k in zip(x,self.sgd_f)])
        return y
    
    def add_gaussian_noise_np(self, np_array, mean=0, std=1):
        noise = np.random.normal(mean, std, np_array.shape)
        self.data = (np_array + noise)/10
        
    def train_generate_data(self):
        DATA = random.sample(Train,1)
        self.add_gaussian_noise_np(DATA[0][0], mean = 0, std = 0.0005)
        a = 2/(sum(self.data*self.data)+1)
        Ima = np.append(self.data*a,a-1)
        return [Ima[:],DATA[0][1]]
    
    def adam_update(self, grad, t, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.m = beta1 * self.m + (1 - beta1) * grad
        self.v = beta2 * self.v + (1 - beta2) * (grad ** 2)
        m_corrected = self.m / (1 - beta1 ** t)
        v_corrected = self.v / (1 - beta2 ** t)
        self.new_grad = lr * m_corrected / (np.sqrt(v_corrected) + epsilon)
        return 0

        
    def algorithm(self,X,Y,epoch):
        gamma_n = self.gamma_0*(epoch**(-self.t))
        #下面是导数，对于图像分类要换掉
        #z = -Y*self.comput_f_hat(X)
        #coe = gamma_n*np.exp(z)*(Y)/(1+np.exp(z))
        F = self.comput_F_hat(X)
        COE = gamma_n*(Y-F)
        if epoch**self.theta >= self.dimPiLn:
            self.order = self.order + 1
            self.update_Z_nt()
            if self.order == 1:
                self.dimPiLn = self.d + 1
                for i in range(10):
                    self.F_hat[i].append(np.zeros(self.d))
                    self.F_bar[i].append(np.zeros(self.d))
            elif self.order == 2:
                self.dimPiLn = (d+3)*d/2
                for i in range(10):
                    self.F_hat[i].append(np.zeros((self.d,self.d)))
                    self.F_bar[i].append(np.zeros((self.d,self.d)))
        self.compute_Znt(X)
        Epoch = epoch+1
        for i in range(10):
            coe = COE[i]
            self.F_hat[i] = [ a + coe*b for a,b in zip(self.F_hat[i],self.ZntX) ][:]
            self.F_bar[i] = [ (a + epoch*b)/Epoch for a,b in zip(self.F_hat[i],self.F_bar[i])][:]
        #error = np.log(1+np.exp(z))
        return 0#error 
    
    def algorithm_sgd(self,X,Y,epoch):
        gamma_n = self.gamma_0*(epoch**(-self.t))
        coe = gamma_n*(Y-self.comput_f_sgd(X))
        self.sgd_f = [j + coe*k for j,k in zip(self.sgd_f,X)]

    def test(self,T,epoch):
        Error_last = 0
        Error_ave = 0
        Accac_last = 0
        Accac_ave = 0
        
        for k in range(T):
            Z = Test_data[k]
            #这里误差衡量也要改
            f_hat =self.comput_F_hat(Z[0])
            t_last = np.where(Z[1] == max(Z[1]))==np.where(f_hat==max(f_hat))
            f_ave = self.comput_F_bar(Z[0])
            t_ave =  np.where(Z[1] == max(Z[1]))==np.where(f_ave==max(f_ave))
            #e_ave = np.log(1+np.exp(-t_ave))
            #e_last = np.log(1+np.exp(-t_last))
            e_ave = sum(np.square(f_ave-Z[1]))
            e_last = sum(np.square(f_hat-Z[1]))
            Error_ave += e_ave
            Error_last += e_last
            if t_ave > 0:
                Accac_ave += 1
            if t_last > 0:
                Accac_last += 1
        self.ERROR_last.append(Error_last/T)
        self.ERROR_ave.append(Error_ave/T)
        self.Accuracy_ave.append(Accac_ave/T)
        self.Accuracy_last.append(Accac_last/T)
        print(epoch,self.ERROR_last[-1], self.ERROR_ave[-1],self.Accuracy_last[-1],self.Accuracy_ave[-1])
        
    def update(self,n,operation):
        self.Accuracy_last = []
        self.Accuracy_ave = []
        self.ERROR_last = []
        self.ERROR_ave = []
        self.Time = []
        start = time.perf_counter()
        T = len(Test_data) #测试数据的多少
        for i in range(1,n+1):
            epoch = i
            DATA = self.train_generate_data()
            error = self.algorithm(DATA[0],DATA[1],epoch)
            '''if error < np.log(2):
                Acc += 1
            Error += error'''
            if epoch in self.N:
                if operation == 'time':
                    end = time.perf_counter()
                    runTime = end - start
                    self.Time.append(runTime)
                    print(epoch, runTime)
                elif operation == 'test':    
                    print(epoch)
                    self.test(T,epoch)
        return [self.ERROR_last,self.ERROR_ave,self.Accuracy_last,self.Accuracy_ave]

In [6]:
d = 785
gamma_0 = 0.5
theta = 0.68
t = 0.
s = 0.505
N = [int(10**(i/5))for i in range(1,36)]
A = [[0 for  i in N] for j in range(6)]
for i in range(1):
    t_kernel_SGD = T_kernel_SGD(d, gamma_0+i*10**(-8), theta, t, s,N)
    B = t_kernel_SGD.update(1600004,'test')
'''You can test the accuracy of the program on the test set or test the code run time by setting "test" or "time" on the previous line'''

1
1 1.3433620754752407 1.0460529315348073 0.1009 0.1009
2
2 1.339303799751153 1.0402830048686371 0.0892 0.1012
3
3 1.333970206565372 1.0079826159038185 0.098 0.1186
6
6 1.4099907373393719 1.0617361248974895 0.0892 0.1628
10
10 1.2962220508373843 0.9611169126874053 0.101 0.1981
15
15 1.1493813185172037 0.884069676730407 0.1158 0.3069
25
25 1.2659405810603963 0.8222937917699341 0.106 0.3406
39
39 1.0625393964562297 0.7767727994223752 0.1628 0.4683
63
63 0.8729151780738202 0.7150696722023557 0.3164 0.5758
100
100 0.7657568911248904 0.6553496282210676 0.4164 0.6848
158
158 0.7333029028771004 0.5957093405776389 0.5421 0.7188
251
251 0.6992188633091834 0.5422951739101578 0.6182 0.7563
398
398 0.7879842080290176 0.49775314469582715 0.5669 0.7921
630
630 0.653025405252053 0.4652021959236656 0.7052 0.811
1000
1000 0.5711605978571026 0.4396214631620695 0.7466 0.8324
1584
1584 0.7421631029351837 0.4244615077016383 0.6978 0.8436
2511
2511 0.5268124902830876 0.4126349675918307 0.7936 0.8527
3981
39

KeyboardInterrupt: 